<a href="https://colab.research.google.com/github/HssngH/project/blob/main/Basic_analysis_with_Python_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (8,123 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 155229 files and d

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
plt.rc('font',family='NanumGothic')
mpl.rcParams['axes.unicode_minus']=False

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## 데이터 읽기

* `welfare_python.csv`

variable | detail | code
:--- | :--- | :--- 
gender | 성별 | 1=남, 2=여
birth | 태어난 연도 | YYYY
marriage | 혼인상태 | 0=비해당(18세미만), 1=배우자있음, 2=사별, 3=이혼, 4=별거, 5=미혼(18세이상, 미혼모포함), 6=기타(사망 등)
religion | 종교 유무 | 1=있음, 2=없음
code_job | 직업 코드 | float64(xxx) (`welfare_job.csv` 참고)
income | 소득(월평균 임금) | float64(xxx)
code_region | 7개 권역별 지역구분 | 1=서울, 2=수도권(인천/경기), 3=부산/경남/울산, 4=대구/경북, 5=대전/충남, 6=강원/충북, 7=광주/전남/전북/제주



In [ ]:
#data path
dpath = '/content/drive/MyDrive/Colab Notebooks/통프2/'

In [ ]:
df = pd.read_csv(dpath+"welfare_python.csv")
df.head()

,gender,birth,marriage,religion,code_job,income,code_region
0,2,1936,2,2,NaN,NaN,1
1,2,1945,2,2,NaN,NaN,1
2,1,1948,2,2,942.0,120.0,1
3,1,1942,3,1,762.0,200.0,1
4,2,1923,2,1,NaN,NaN,1


In [ ]:
# unicode error 뜨는 경우 encoding="CP949" 옵션 추가
ref = pd.read_csv(dpath+"welfare_job.csv",encoding="CP949")
ref.head()

,code_job,job
0,111,의회의원 고위공무원 및 공공단체임원
1,112,기업고위임원
2,120,행정 및 경영지원 관리자
3,131,연구 교육 및 법률 관련 관리자
4,132,보험 및 금융 관리자


## 데이터 전처리

1. gender : 1이면 '남성', 2이면 '여성'으로 변경하여 gender2로 코딩
2. marriage : 1이면 '결혼', 3이면 '이혼', 나머지는 '그외'로 변경하여 marriage2로 코딩
3. code_job : `welfare_job.csv'에서 한글로 된 직업이름정보(job)를 추출하여 병합하여 job으로 코딩
4. code_region : 7개 권역별 지역이름을 문자열로 변경하여 region으로 변경
5. age : 현재 연도에서 태어난 연도를 빼서 신규컬럼 생성
6. religion : 1이면 '종교있음', 2이면 '무교'로 religion2로 표기
7. income : 소득이 없는 결측치의 경우 다른 월급들의 평균으로 대체 income2로 저장
8. agegroup : 연령(age)이 30세 미만이면 '청년', 30세 이상이면서 60세 미만은 '중년', 60세 이상이면 '노년'으로 신규컬럼 코딩 후 범주형 변수로 변경하기

In [ ]:
# 1. gender : 1이면 '남성', 2이면 '여성'으로 gender2로 코딩 (loc메소드 적용)
df["gender2"]=["남성" if i==1 else "여성" for i in df.gender]
df

,gender,birth,marriage,religion,code_job,income,code_region,gender2
0,2,1936,2,2,NaN,NaN,1,여성
1,2,1945,2,2,NaN,NaN,1,여성
2,1,1948,2,2,942.0,120.0,1,남성
3,1,1942,3,1,762.0,200.0,1,남성
4,2,1923,2,1,NaN,NaN,1,여성
...,...,...,...,...,...,...,...,...
16659,2,1967,1,1,NaN,NaN,5,여성
16660,2,1992,5,1,314.0,302.5,5,여성
16661,1,1995,5,1,NaN,NaN,5,남성
16662,2,1998,0,1,NaN,NaN,5,여성


In [ ]:
# 2. marriage : 1이면 '결혼', 3이면 '이혼', 나머지는 '그외'라는 문자열로 처리하여 marriage2로 저장 (lambda함수 적용해보기)
def recoding(x):
    if x==1:
        return "결혼"
    if x==3:
        return "이혼"
    else:
        return "그외"


df["marriage2"]=df.marriage.apply(lambda x : recoding(x))
df.head()

,gender,birth,marriage,religion,code_job,income,code_region,gender2,marriage2
0,2,1936,2,2,NaN,NaN,1,여성,그외
1,2,1945,2,2,NaN,NaN,1,여성,그외
2,1,1948,2,2,942.0,120.0,1,남성,그외
3,1,1942,3,1,762.0,200.0,1,남성,이혼
4,2,1923,2,1,NaN,NaN,1,여성,그외


In [ ]:
# 3. code_job : `welfare_job.csv'에서 한글로 된 직업이름정보(job)를 추출하여 병합 (merge이용)
ref.head()
df=pd.merge(df,ref,on="code_job")
df

,gender,birth,marriage,religion,code_job,income,code_region,gender2,marriage2,job
0,1,1948,2,2,942.0,120.0,1,남성,그외,경비원 및 검표원
1,1,1945,3,2,942.0,220.2,1,남성,이혼,경비원 및 검표원
2,1,1946,1,2,942.0,139.0,1,남성,결혼,경비원 및 검표원
3,1,1953,1,2,942.0,150.0,1,남성,결혼,경비원 및 검표원
4,1,1960,1,1,942.0,166.0,1,남성,결혼,경비원 및 검표원
...,...,...,...,...,...,...,...,...,...,...
7524,2,1950,1,1,819.0,NaN,6,여성,결혼,기타 식품가공관련 기계조작원
7525,1,1960,1,1,111.0,250.0,7,남성,결혼,의회의원 고위공무원 및 공공단체임원
7526,1,1960,1,2,111.0,1250.0,1,남성,결혼,의회의원 고위공무원 및 공공단체임원
7527,1,1992,5,1,876.0,280.0,3,남성,그외,선박 갑판승무원 및 관련 종사원


In [ ]:
# 4. code_region : 7개 권역별 지역이름을 문자열로 표기하여 region에 저장 (함수 적용)
# 1=서울, 2=인천/경기, 3=부산/경남/울산, 4=대구/경북, 5=대전/충남, 6=강원/충북, 7=광주/전남/전북/제주
df.code_region.value_counts()
def recoding(x):
    if x == 1:
        return '서울'
    elif x ==2:
        return '인천/경기'
    elif x ==3:
        return '부산/경남/울산'
    elif x ==4:
        return '대구/경북'
    elif x ==5:
        return '대전/충남'
    elif x ==6:
        return '강원/충북'
    else:
        return '광주/전남/전북/제주'

df["region"]=df.code_region.apply(recoding)
df

,gender,birth,marriage,religion,code_job,income,code_region,gender2,marriage2,job,region
0,1,1948,2,2,942.0,120.0,1,남성,그외,경비원 및 검표원,서울
1,1,1945,3,2,942.0,220.2,1,남성,이혼,경비원 및 검표원,서울
2,1,1946,1,2,942.0,139.0,1,남성,결혼,경비원 및 검표원,서울
3,1,1953,1,2,942.0,150.0,1,남성,결혼,경비원 및 검표원,서울
4,1,1960,1,1,942.0,166.0,1,남성,결혼,경비원 및 검표원,서울
...,...,...,...,...,...,...,...,...,...,...,...
7524,2,1950,1,1,819.0,NaN,6,여성,결혼,기타 식품가공관련 기계조작원,강원/충북
7525,1,1960,1,1,111.0,250.0,7,남성,결혼,의회의원 고위공무원 및 공공단체임원,광주/전남/전북/제주
7526,1,1960,1,2,111.0,1250.0,1,남성,결혼,의회의원 고위공무원 및 공공단체임원,서울
7527,1,1992,5,1,876.0,280.0,3,남성,그외,선박 갑판승무원 및 관련 종사원,부산/경남/울산


In [ ]:
# 4. code_region : 7개 권역별 지역이름을 문자열로 표기하여 region에 저장 (dictionary 적용)
# 1=서울, 2=인천/경기, 3=부산/경남/울산, 4=대구/경북, 5=대전/충남, 6=강원/충북, 7=광주/전남/전북/제주
dic = { 1: '서울', 2:'인천/경기',3:'부산/경남/울산',4:'대구/경북',5:'대전/충남',6:'강원/충북',7:'광주/전남/전북/제주'}
df["region"]=[dic[i] for i in df.code_region]
df.head()

,gender,birth,marriage,religion,code_job,income,code_region,gender2,marriage2,job,region
0,1,1948,2,2,942.0,120.0,1,남성,그외,경비원 및 검표원,서울
1,1,1945,3,2,942.0,220.2,1,남성,이혼,경비원 및 검표원,서울
2,1,1946,1,2,942.0,139.0,1,남성,결혼,경비원 및 검표원,서울
3,1,1953,1,2,942.0,150.0,1,남성,결혼,경비원 및 검표원,서울
4,1,1960,1,1,942.0,166.0,1,남성,결혼,경비원 및 검표원,서울


In [ ]:
# 5. age : 현재 연도에서 태어난 연도를 빼서 신규컬럼 생성
df["age"]=2022-df["birth"]
df.head()

,gender,birth,marriage,religion,code_job,income,code_region,gender2,marriage2,job,region,age
0,1,1948,2,2,942.0,120.0,1,남성,그외,경비원 및 검표원,서울,74
1,1,1945,3,2,942.0,220.2,1,남성,이혼,경비원 및 검표원,서울,77
2,1,1946,1,2,942.0,139.0,1,남성,결혼,경비원 및 검표원,서울,76
3,1,1953,1,2,942.0,150.0,1,남성,결혼,경비원 및 검표원,서울,69
4,1,1960,1,1,942.0,166.0,1,남성,결혼,경비원 및 검표원,서울,62


In [ ]:
# 5. age : 현재 연도에서 태어난 연도를 빼서 신규컬럼 생성 (현재 연도를 함수로 가져와보기)
# 구글에서 'year in python' 검색
import datetime
df_now=datetime.datetime.now()
df_now.year

df["age"]=df_now.year - df.birth
df["age"]=datetime.datetime.now().year-df.birth
df.head()

,gender,birth,marriage,religion,code_job,income,code_region,gender2,marriage2,job,region,age
0,1,1948,2,2,942.0,120.0,1,남성,그외,경비원 및 검표원,서울,74
1,1,1945,3,2,942.0,220.2,1,남성,이혼,경비원 및 검표원,서울,77
2,1,1946,1,2,942.0,139.0,1,남성,결혼,경비원 및 검표원,서울,76
3,1,1953,1,2,942.0,150.0,1,남성,결혼,경비원 및 검표원,서울,69
4,1,1960,1,1,942.0,166.0,1,남성,결혼,경비원 및 검표원,서울,62


In [ ]:
# 6. religion : 1이면 '종교있음', 2이면 '무교'로 표기하여 religion2로 코딩 (if else 구문 이용)
df["religion2"]= ["유" if i==1 else "무" for i in df.religion]
df.head()

,gender,birth,marriage,religion,code_job,income,code_region,gender2,marriage2,job,region,age,religion2
0,1,1948,2,2,942.0,120.0,1,남성,그외,경비원 및 검표원,서울,74,무
1,1,1945,3,2,942.0,220.2,1,남성,이혼,경비원 및 검표원,서울,77,무
2,1,1946,1,2,942.0,139.0,1,남성,결혼,경비원 및 검표원,서울,76,무
3,1,1953,1,2,942.0,150.0,1,남성,결혼,경비원 및 검표원,서울,69,무
4,1,1960,1,1,942.0,166.0,1,남성,결혼,경비원 및 검표원,서울,62,유


In [ ]:
# 7. income : 소득이 없는 결측치의 경우 다른 월급들의 평균으로 대체하여 income2로 코딩


In [ ]:
# 8. agegroup : 연령(age)이 30세 미만이면 '청년', 30세 이상이면서 60세 미만은 '중년', 60세 이상이면 '노년'으로 신규컬럼 코딩 (함수 이용)


In [ ]:
# 8. 범주형 변수로 순서 카테고리 지정하기


In [ ]:
# 9. 다음 변수만 모아서 한글변수명을 붙이고 df2에 저장
# age=연령, agegroup=연령대, birth=생년, gender2=성별, marriage2=결혼상태, religion2=종교유무, 
# region=지역, job=직업, income2=월소득


In [ ]:
# 위치가 변경될 수 있으니 dictionary 이용


In [ ]:
# df2를 /contents/drive/MyDrive/data/output/welfare_clean.csv로 저장


In [ ]:
# welfare_clean.csv를 불러와서 잘 저장되었는지 확인해보기


In [ ]:
# df2를 /contents/drive/MyDrive/data/output/welfare_clean.csv로 저장 (index 없이 저장하기)


# welfare_clean.csv를 불러와서 잘 저장되었는지 확인해보기


## 데이터 시각화

### 결혼상태와 종교유무의 분포

In [ ]:
# 결혼상태별 빈도 확인


In [ ]:
# 빈도막대그래프 그리기


In [ ]:
# 결혼상태 순서를 정해주기


In [ ]:
# countplot()의 order 옵션 사용해서 이혼, 결혼, 그외 순으로 그림 그려보기


In [ ]:
# 결혼상태에 따라 종교유무를 나누어 표기
# 막대 색상을 팔레트 "Set2" 이용


In [ ]:
# 결혼상태에 따라 종교유무를 나누어 표기 (종교 유 > 무 순으로 표기)
# 테두리 두께를 1로 지정하고, 테두리 색상을 navy로 주기


In [ ]:
# 위 그림을 가로막대로 그리고, 막대 색상을 팔레트 "Paired"로 주고, 테두리 색상을 gray로 주기


### 연령 분포


In [ ]:
# 연령에 대한 히스토그램


In [ ]:
# 연령에 대한 히스토그램에 밀도곡선 얹고, 색상을 magenta로 지정하기


### 연령과 소득, 결혼유무 분포

In [ ]:
# 연령과 월소득의 산점도


In [ ]:
# 연령과 월소득의 산점도 그린 뒤, 마진에 히스토그램 얹기


In [ ]:
# pairplot으로 그림 그리기


In [ ]:
# pairplot의 대각선은 밀도곡선으로 변경하고, 하삼각만 그리기


In [ ]:
# 연령과 월소득에 대한 pairplot을 결혼상태에 따라 색상을 구분하여 그리기


In [ ]:
# 연령과 월소득에 대한 산점도를 결혼상태에 따라 나눠서 그리되, 결혼상태는 결혼과 이혼만 추출해서 그리기


In [ ]:
# 연령에 따른 월소득을 회귀도표로 나타내되, 결혼상태에 따라 항목을 나누어서 그리고, 90% 신뢰구간 추가하기


### 성별과 연령 분포

In [ ]:
# 연령에 대한 박스그림 


In [ ]:
# 성별에 따라 연령에 대한 바이올린 그림 그리기


In [ ]:
# 성별에 따른 연령분포에 대한 바이올린 그림을 결혼상태에 따라 나눠 그리기 (단, 결혼상태는 '결혼', '그외'만 추출하여 그리기)


In [ ]:
# 성별에 따른 연령분포에 대한 바이올린 그림을 결혼상태에 따라 분할된 바이올린으로 그리기 (단, 결혼상태는 '결혼', '그외'만 추출하여 그리기)


### 성별과 소득 분포

In [ ]:
# 성별에 따른 평균 월소득 계산


In [ ]:
# 성별에 따른 평균 월소득에 대한 평균막대그래프 (신뢰구간 표시 안하기)


In [ ]:
# 성별에 따른 평균 월소득에 대한 평균막대그래프를 결혼상태에 따라 구분하여 가로막대로 그리기


In [ ]:
# 성별에 따른 월소득에 대한 박스그림 그리기


In [ ]:
# 성별에 따른 월소득에 대한 박스그림 그리기 (평균 월소득이 400 이하인 개체만 추출)


In [ ]:
# 위 그림에 jitter형태로 데이터 표현하기 (색상은 검정, 점모양은 .으로 표현)


# 서울시 구별 CCTV 현황 분석

* [서울시 자치구 연도별 CCTV 설치 현황 데이터](http://data.seoul.go.kr/dataList/OA-2734/F/1/datasetView.do)
* [서울시 자치구 연도별 인구 데이터](https://data.seoul.go.kr/dataList/10790/S/2/datasetView.do)
* 서울시 구별 CCTV 설치현황(구별 인구대비현황 등) 분석
* source: [파이썬으로 데이터 주무르기](https://github.com/PinkWink/DataScience)



## 데이터 읽기

In [ ]:
# CCTV 데이터 읽어오기

In [ ]:
# 구분이 "계"인 행 삭제


In [ ]:
# 2021년 열 삭제


In [ ]:
# 인구 데이터 읽어오기


In [ ]:
# 자치구가 "합계"인 행 삭제


In [ ]:
# 자치구별 데이터 개수 확인


In [ ]:
# 연도 최소 최대 확인


## 데이터 파악



### CCTV 현황

In [ ]:
# CCTV 데이터에서 CCTV 전체개수로 정렬해서 CCTV가 가장 적은 구부터 5개구 확인하기


In [ ]:
# CCTV 데이터에서 CCTV 전체개수로 정렬해서 CCTV가 가장 많은 구부터 5개구 확인하기


In [ ]:
# 최근 3년간 CCTV 증가율 계산 (최근 3년간 CCTV 수를 더하고, 최근 3년 이전 CCTV 수로 나누기) 


In [ ]:
# 최근 3년 CCTV 증가율이 가장 높았던 구부터 5개구 확인


### 서울시 인구현황

In [ ]:
# 구별 외국인비율과 고령자비율 계산


In [ ]:
# 2020년 기준 인구수가 가장 많은 구부터 5개구 확인


In [ ]:
# 2020년 기준 외국인 비율이 가장 높은 구부터 5개구 확인


In [ ]:
# 2020년 기준 고령자 비율이 가장 높은 구부터 5개구 확인


## 데이터 병합

In [ ]:
# CCTV 데이터에서 불필요한 부분 삭제


In [ ]:
# 서울시 데이터에서 불필요한 부분 삭제 (2020년 기준 자치구, 인구수, 한국인, 외국인, 고령자, 외국인비율, 고령자비율만 남기기)


In [ ]:
# 2020년 데이터만 남기기


In [ ]:
# 두 데이터 병합 (구 이름 기준)


In [ ]:
# 구 이름을 index로 부여하기


## 데이터 분석

### 상관분석


In [ ]:
# CCTV수와 인구수, 고령자비율, 외국인비율 간의 상관관계
# ?pd.DataFrame.corr


In [ ]:
# heatmap으로 나타내기


### 구별 CCTV 현황 그래프

In [ ]:
# 구별 CCTV 총계를 가로막대그래프로 그려보기


In [ ]:
# 총계가 가장 높은 구부터 차례로 도표


In [ ]:
# 인구수 대비 CCTV 개수의 비율을 구하고 가장 높은 구부터 도표


### 인구수와 CCTV 관계

In [ ]:
# 인구수 대비 CCTV 수에 대한 산점도 그리기


In [ ]:
# 인구수 대비 CCTV 수에 대한 산점도 그리기 (CCTV가 가장 많은 구부터 5개구만 표기하기)


In [ ]:
# 인구수 대비 CCTV 개수에 대한 회귀도표 그리고 90% 신뢰구간 표기하기


# 일자리와 부동산 관계 분석

* [지역별 고용노동통계 데이터](http://laborstat.moel.go.kr/hmp/index.do)
* [KOSIS 주민등록세대수](https://kosis.kr/)
* [KOSIS 아파트 실거래 가격 지수](https://kosis.kr/)

* source: [파이썬으로 데이터 주무르기](https://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791158391546)


## 데이터 읽기


In [ ]:
# 지역별 일자리 데이터 읽어오기


In [ ]:
# 지역명 나누기


In [ ]:
# 세대수 데이터


In [ ]:
## 아파트 매매가격지수


## 데이터 병합

## 데이터 분석


In [ ]:
# 시도별 고용자수를 막대그래프로 그리기


In [ ]:
# 고용자수 높은 시도부터 도표


In [ ]:
# 고용자수 높은 시도부터 도표 (평균 고용자수 라인 추가)


In [ ]:
# 세대수 대비 고용비율 컬럼 추가


In [ ]:
# 시도 단위 세대수 대비 고용비율 도표


In [ ]:
# 세대수 대비 고용률과 아파트 매매가지수 회귀도표


In [ ]:
# 시도별 고용자수, 세대수를 아파트 매매가지수와 함께 살펴보기


In [ ]:
# 시도별 세대수 대비 고용률, 세대수를 아파트 매매가지수와 함께 살펴보기 ()


# THE END